In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
epochs = 100
batch_size = 32
learning_rate = 0.0001
momentum = 0.2

In [2]:
#Data preparation
class CustomImageDataset(Dataset):
    def __init__(self, labels_arr, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.DataFrame(labels_arr)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = torch.tensor(int(self.img_labels.iloc[idx, 1]))
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    
def create_dataset(folder, label):
    files = os.listdir(folder)
    for i in files:
        if i =='.DS_Store':
            files.remove(i)
    files = np.array(files)
    labels_arr = np.zeros(len(files), dtype=np.uint8)
    labels_arr.fill(label)
    labels_arr = np.column_stack((files, labels_arr))

    transform = transforms.Compose([transforms.Resize((140, 140))])
    dataset = CustomImageDataset(labels_arr, folder, transform=transform)
    return dataset

#dataloader_low = create_dataset("/Users/gaky/Desktop/efir/low", int(2))
#ataloader_medium = create_dataset("/Users/gaky/Desktop/efir/medium", int(3))
dataloader_high = create_dataset("/Users/gaky/Desktop/efir/high_small", int(0))
dataloader_off = create_dataset("/Users/gaky/Desktop/efir/off", int(1))

#dataloader_low_blur = create_dataset("/Users/gaky/Desktop/efir/low blur", int(0))
#dataloader_medium_blur = create_dataset("/Users/gaky/Desktop/efir/medium blur", int(1))
dataloader_high_blur = create_dataset("/Users/gaky/Desktop/efir/high_small", int(0))
dataloader_off_blur = create_dataset("/Users/gaky/Desktop/efir/off blur", int(1))

train_dev_sets = ConcatDataset([dataloader_high, 
                                dataloader_off, dataloader_high_blur, dataloader_off_blur])
train_dev_off = ConcatDataset([dataloader_off,  dataloader_off_blur])

dataloader = DataLoader(train_dev_sets, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(train_dev_sets, batch_size=len(train_dev_sets), shuffle=True)
dataloader_test_off = DataLoader(train_dev_off, batch_size=len(train_dev_off), shuffle=True)

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, stride=1, padding=1)#140x140
        nn.init.kaiming_normal_(self.conv1.weight, mode='fan_in', nonlinearity='relu')
        self.conv12 = nn.Conv2d(10, 10, 3, stride=1, padding=1)#140x140
        
        self.pool1 = nn.MaxPool2d(2, stride=2)#70x70
        
        self.conv2 = nn.Conv2d(10, 20, 3, stride=1, padding=1)#70x70
        nn.init.kaiming_normal_(self.conv2.weight, mode='fan_in', nonlinearity='relu')
        
        self.pool2 = nn.MaxPool2d(2, stride=2)#35x35
        
        self.conv3 = nn.Conv2d(20, 30, 3, stride=1, padding=1)#35x35
        nn.init.kaiming_normal_(self.conv3.weight, mode='fan_in', nonlinearity='relu')
        
        self.pool3 = nn.MaxPool2d(2, stride=2)#16x16
        
        self.conv4 = nn.Conv2d(30, 30, 3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv4.weight, mode='fan_in', nonlinearity='relu')
        
        self.pool4 = nn.MaxPool2d(2, stride=2)
        
        self.conv5 = nn.Conv2d(20, 20, 3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv4.weight, mode='fan_in', nonlinearity='relu')
        
        self.pool5 = nn.MaxPool2d(2, stride=2)
        

        self.fc1 = nn.Linear(4*4*30, 50)
        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        
        self.fc2 = nn.Linear(50, 30)
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
        
        self.fc3 = nn.Linear(30, 30)
        nn.init.kaiming_normal_(self.fc3.weight, mode='fan_in', nonlinearity='relu')
        
        self.fc4 = nn.Linear(30, 2)
        nn.init.kaiming_normal_(self.fc4.weight, mode='fan_in', nonlinearity='relu')
        

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv12(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        
        x = self.pool2(x)
            
        x = self.conv3(x)
        x = F.relu(x)
        

        x = self.pool3(x)
        
        x = self.conv4(x) 
        x = F.relu(x)

        x = self.pool3(x)
        
        x = self.pool3(x)
        

        
        #x = self.pool3(x)
    
        
        #x = self.conv4(x)
        #x = F.relu(x)
        
        #x = self.pool4(x)
        
        #x = self.conv5(x)
       # x = F.relu(x)
        
       # x = self.pool5(x)
        
        x = x.view(-1, 4*4*30)   
        
        
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x)
        
        x = self.fc3(x)
        x = F.relu(x)
        x = F.dropout(x)
        
        x = self.fc4(x)
        return x


model = ConvNet().to(device)
#0,4250
#0,575
#weight = [0,2875, 0,7125]
weights = np.array([4.5, 5.5])

weights = torch.tensor(weights).float()


loss_function = nn.CrossEntropyLoss()
#optim = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
contin = True
if contin:
    for epoch in range(epochs):
        for i, (x, y) in enumerate(dataloader):

            x = x/255.
            x = x.to(device)
            y = y.to(device)

            model.zero_grad()
            x = F.normalize(x.float())
            out = model(x.float())

            loss = loss_function(out, y.long())
            loss = loss.mean()
            loss.backward()
            optim.step()

            if epoch % 5 == 0:
                #logic for saving model
                pass

            if epoch % 2 == 0 and i==0:
                dataset_test = iter(dataloader_test)
                x, y = next(dataset_test)
                x = x/255.
                x = x.to(device)
                y = y.to(device)

                with torch.no_grad():
                    out = model(x.float())
                    cat = torch.argmax(out, dim=1)
                    accuracy = (cat == y.long()).float().mean()
                    print(epoch, i)
                    print("accuracy", accuracy)
                    if accuracy.item() > 92:
                        contin = False
                    #print(out)
                    #print(y.long())
            
            if epoch % 5 == 0 and i==0:
                dataset_test = iter(dataloader_test_off)
                x, y = next(dataset_test)
                x = x/255.
                x = x.to(device)
                y = y.to(device)

                with torch.no_grad():
                    out = model(x.float())
                    cat = torch.argmax(out, dim=1)
                    accuracy = (cat == y.long()).float().mean()
                    print(epoch, i)
                    print("off", accuracy)
                    if accuracy.item() > 92:
                        contin = False

            print(epoch, i)
            #print(f'Epoch: {epoch}, Accuracy: {accuracy}', y, cat)


0 0
accuracy tensor(0.4113)
0 0
off tensor(0.1380)
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
2 0
accuracy tensor(0.4472)
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
4 0
accuracy tensor(0.4019)
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
5 0
off tensor(0.2209)
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
6 0
accuracy tensor(0.4604)
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
8 0
accuracy tensor(0.4302)
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
9 16
10 0
accuracy tensor(0.4698)
10 0
off tensor(0.4018)
10 0
10 1
1

KeyboardInterrupt: 

In [15]:
from PIL import Image
import os 
shit = 0

folder = "/Users/gaky/Desktop/efir/high_blur_small/"
list_im = os.listdir(folder)

for i in list_im:
    if i == ".DS_Store":
        list_im.remove(i)

for i in list_im:
    
    image = folder+i
    image = Image.open(image)
    image = image.resize((140, 140), Image.Resampling.LANCZOS)
    image = Image.Image.getdata(image)

    image = np.array(image)

    image = image.reshape(140, 140, 3)
    image = np.transpose(image, (2, 0, 1))
    image = torch.tensor(image).float()


    with torch.no_grad():
        out = model(image)
    cat = torch.argmax(out, dim=1)
    if cat == 0:
        shit+=1

In [16]:
shit/len(list_im)

0.5964912280701754

In [22]:
image = "/Users/gaky/Downloads/web-shots/M0001/13/name__M0001, time__2023-09-06 13_29_20.197961.jpg"

In [23]:
from PIL import Image
image = Image.open(image)
image = image.resize((140, 140), Image.Resampling.LANCZOS)
image = Image.Image.getdata(image)

image = np.array(image)

image = image.reshape(140, 140, 3)
image = np.transpose(image, (2, 0, 1))
image = torch.tensor(image).float()


with torch.no_grad():
    out = model(image)
cat = torch.argmax(out, dim=1)

IsADirectoryError: [Errno 21] Is a directory: '/Users/gaky/Desktop/efir/low/'

In [21]:
cat

tensor([0])

In [7]:
image = read_image("/Users/gaky/Desktop/efir/off/ffd7c47e-1329-4865-84bf-c5340e7225b8.jpg")

In [67]:
type(image)

torch.Tensor

In [68]:
image.shape

torch.Size([3, 720, 1280])

In [70]:
torch.max(image)

tensor(255, dtype=torch.uint8)

In [23]:
dataset_test = iter(dataloader_test)
x, y = next(dataset_test)
x = x/255.
x = x.to(device)
y = y.to(device)

with torch.no_grad():
    out = model(x.float())
    cat = torch.argmax(out, dim=1)
    accuracy = (cat == y.long()).float().mean()
    print("accuracy", accuracy)


accuracy tensor(0.9062)
